In [1]:
import pandas as pd
import numpy as np
import textstat
from sentence_transformers import SentenceTransformer
import scipy
import nltk
import spacy
import sklearn
import jinja2
import faiss
import chromadb
import torch

print("Todas las dependencias están instaladas correctamente.")

Todas las dependencias están instaladas correctamente.


In [2]:
import pandas as pd
from textstat import textstat
from sentence_transformers import SentenceTransformer
import numpy as np
import sys, pandas, textstat, csv, pathlib
print(sys.executable)
print("pandas", pandas.__version__)


c:\Users\jorge\Desktop\TFG\ServicioFlaskGPT\.venv\Scripts\python.exe
pandas 2.2.3


In [6]:
import sys
print(sys.executable)

c:\Users\jorge\Desktop\TFG\ServicioFlaskGPT\.venv\Scripts\python.exe


CASO PARA REQUISITOS DEL EXPERTO

CALCULO DE MEDIAS Y DEMAS

COSAS DEL NOTEBOOK ANTERIOR

In [ ]:
df = pd.read_csv("data/quick_scan_rege.csv", sep=';', quotechar='"')
print(df.head(3))
print(df.shape)        # debería dar (155, 4)



# Verificar que el modelo de spaCy esté instalado
import spacy
try:
    nlp = spacy.load('en_core_web_sm')
    print("Modelo 'en_core_web_sm' cargado correctamente.")
except OSError:
    print("Error: Modelo 'en_core_web_sm' no encontrado. Instálalo con 'python -m spacy download en_core_web_sm'")
    raise

# Preparar modelo y cargar módulos
model = SentenceTransformer('all-MiniLM-L6-v2')


# Cargar el archivo CSV desde la subcarpeta data
df = pd.read_csv('data/req_ia/quick_scan_template.csv')  # Ajusta la ruta si es necesario
requisitos = df['descripcion'].tolist()  # Usamos solo la columna 'descripcion' para embeddings

# Mostrar los primeros 5 requisitos de forma clara
print("Primeros 5 requisitos cargados:")
for i, req in enumerate(requisitos[:5], 1):  # Mostrar solo los primeros 5
    print(f"{i}. {req}")

# Contar el número total de requisitos (excluyendo el encabezado)
total_requisitos = len(df)  # pandas ya excluye el encabezado al contar las filas
print(f"\nTotal de requisitos procesados: {total_requisitos}")



# Generar embeddings para los requisitos
embeddings = model.encode(requisitos, convert_to_tensor=True)
print(f"Forma de los embeddings: {embeddings.shape}")



# Calcular la matriz de similitud coseno
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(embeddings.cpu().numpy())
print("Matriz de similitud:")
print(similarity_matrix)

# Guardar la matriz de similitud como archivo .npy (formato binario de NumPy)
np.save('rq5/similarity_matrix.npy', similarity_matrix)
print("Matriz de similitud guardada en 'similarity_matrix.npy'")

# Opcional: Guardar como CSV para inspección manual
pd.DataFrame(similarity_matrix).to_csv('rq5/similarity_matrix.csv', index=False)
print("Matriz de similitud también guardada en 'similarity_matrix.csv'")


# Identificar pares con alta similitud (por ejemplo, >0.7)
import pandas as pd
import numpy as np

# Cargar la matriz de similitud desde el archivo CSV
similarity_matrix = pd.read_csv('rq5/similarity_matrix.csv').values  # Convierte el CSV a un array NumPy
print("Matriz de similitud cargada (forma:", similarity_matrix.shape, ")")

# Cargar los requisitos originales desde el CSV
df = pd.read_csv('data/req_ia/quick_scan_template.csv')
requisitos = df['descripcion'].tolist()
print("Requisitos cargados:", len(requisitos))

# Crear una lista de pares con alta similitud
high_similarity_pairs = []
for i in range(len(requisitos)):
    for j in range(i + 1, len(requisitos)):  # Evitar comparar un requisito consigo mismo (i != j)
        if similarity_matrix[i][j] > 0.7:  # Umbral ajustable (0.7 como ejemplo)
            high_similarity_pairs.append({
                'Req_1_ID': df['requerimiento_id'][i],  # ID del primer requisito
                'Req_1': requisitos[i],
                'Req_2_ID': df['requerimiento_id'][j],  # ID del segundo requisito
                'Req_2': requisitos[j],
                'Similarity': similarity_matrix[i][j]
            })

# Convertir a DataFrame y guardar
if high_similarity_pairs:
    df_high_similarity = pd.DataFrame(high_similarity_pairs)
    df_high_similarity.to_csv('rq5/high_similarity_pairs.csv', index=False)
    print("Pares con alta similitud guardados en 'rq5/high_similarity_pairs.csv':")
    print(df_high_similarity)

    # Mostrar los 3 pares con mayor similitud
    print("\nTop 3 pares con mayor similitud:")
    top_3_pairs = df_high_similarity.sort_values(by='Similarity', ascending=False).head(3)
    print(top_3_pairs)
else:
    print("No se encontraron pares con similitud mayor a 0.7.")


##CELDA PARA VISUALIZACION DE LA MATRIZ DE SIMILITUD
import numpy as np

# Cargar la matriz guardada
similarity_matrix = np.load('rq5/similarity_matrix.npy')
print("Forma de la matriz:", similarity_matrix.shape)
print("Primeros 5x5 elementos:")
print(similarity_matrix[:5, :5])  # Muestra solo una parte para no saturar


# Índice de legibilidad
import textstat
def get_legibility_score(text):
    return textstat.flesch_reading_ease(text)

df['legibility_score'] = df['descripcion'].apply(get_legibility_score)
print("Puntuaciones de legibilidad:")
print(df[['requerimiento_id', 'descripcion', 'legibility_score']])




# Detectar términos ambiguos
import spacy
nlp = spacy.load('en_core_web_sm')
def detect_ambiguous_terms(text):
    doc = nlp(text)
    ambiguous_terms = [token.text for token in doc if token.text.lower() in ['fast', 'quick', 'secure', 'efficient']]
    return ambiguous_terms



df['ambiguous_terms'] = df['descripcion'].apply(detect_ambiguous_terms)
print("Términos ambiguos:")
print(df[['requerimiento_id', 'descripcion', 'ambiguous_terms']])

# Guardar resultados
df.to_csv('checklist_with_analysis.csv', index=False)
print("Análisis guardado en checklist_with_analysis.csv")


##Enfoque final con simlitud entre otros requsitos y legibilidad para decidir calidad de cada requisito
import pandas as pd
import numpy as np

# Cargar la matriz de similitud desde el archivo CSV
similarity_matrix = pd.read_csv('rq5/similarity_matrix.csv').values  # Convierte el CSV a un array NumPy
print("Matriz de similitud cargada (forma:", similarity_matrix.shape, ")")

# Cargar los requisitos originales desde el CSV
df_requisitos = pd.read_csv('rq5/checklist_with_analysis.csv')
requisitos = df_requisitos['descripcion'].tolist()
requerimiento_ids = df_requisitos['requerimiento_id'].tolist()
print("Requisitos cargados:", len(requisitos))

# Calcular la similitud máxima para cada requisito
similitud_maxima_list = []
for i in range(len(requisitos)):
    max_sim = 0.0
    similar_id_idx = None
    for j in range(len(requisitos)):
        if i != j:  # Evitar comparar un requisito consigo mismo
            if similarity_matrix[i][j] > max_sim:
                max_sim = similarity_matrix[i][j]
                similar_id_idx = j
    similar_id = requerimiento_ids[similar_id_idx] if similar_id_idx is not None else "N/A"
    similitud_maxima_list.append(f"{max_sim:.2f}-{similar_id}")

# Añadir la columna de similitud máxima al DataFrame
df_requisitos['similitud_maxima'] = similitud_maxima_list

# Seleccionar solo las columnas requeridas
df_output = df_requisitos[['requerimiento_id', 'descripcion', 'legibility_score', 'similitud_maxima']]

# Guardar el resultado en un nuevo CSV
df_output.to_csv('rq5/requisitos_analisis_simiYlegi.csv', index=False)
print("CSV generado como 'rq5/requisitos_analisis_simiYlegi.csv' con similitud máxima y puntuación de legibilidad.")

##PROCESO DE VALIDACION REQUISITOS EXPERTO

import pandas as pd

# Leer el archivo CSV con coma como separador y comillas como quotechar
df = pd.read_csv("data/req_human/checklist_human.csv", sep=',', quotechar='"')
print(df.head(3))
print(df.shape)


# Cargar el archivo CSV desde la subcarpeta data
df = pd.read_csv('data/req_human/checklist_human.csv')  # Ajusta la ruta si es necesario
requisitos_ex = df['Descripción'].tolist()  # Usamos solo la columna 'descripcion' para embeddings

# Mostrar los primeros 5 requisitos de forma clara
print("Primeros 5 requisitos cargados:")
for i, req in enumerate(requisitos_ex[:5], 1):  # Mostrar solo los primeros 5
    print(f"{i}. {req}")

# Contar el número total de requisitos (excluyendo el encabezado)
total_requisitos_ex = len(df)  # pandas ya excluye el encabezado al contar las filas
print(f"\nTotal de requisitos procesados: {total_requisitos_ex}")


# Generar embeddings para los requisitos
embeddings = model.encode(requisitos_ex, convert_to_tensor=True)
print(f"Forma de los embeddings: {embeddings.shape}")

# Calcular la matriz de similitud coseno
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix_ex = cosine_similarity(embeddings.cpu().numpy())
print("Matriz de similitud requisitos del experto:")
print(similarity_matrix_ex)

# Guardar la matriz de similitud como archivo .npy (formato binario de NumPy)
np.save('data/req_human/results/similarity_matrix_ex.npy', similarity_matrix_ex)
print("Matriz de similitud guardada en 'similarity_matrix_ex.npy'")

# Opcional: Guardar como CSV para inspección manual
pd.DataFrame(similarity_matrix_ex).to_csv('data/req_human/results/similarity_matrix_ex.csv', index=False)
print("Matriz de similitud también guardada en 'similarity_matrix_ex.csv'")
print("Forma de la matriz:", similarity_matrix_ex.shape)

##UNA VEZ TENEMOS LA MATRIZ DE SIMILITUD PARA LOS REQUISITOS DEL EXPERTO 
##PASAMOS A CALCULAR LOS PARES MAS SIMILARES DE DICHA MATRIZ 
##FUSIONARLO CON EL CALCULO DE LA LEGIBILIDAD

import pandas as pd
import numpy as np
import textstat

# Cargar la matriz de similitud desde el archivo .npy (más precisa que CSV)
similarity_matrix = np.load('data/req_human/results/similarity_matrix_ex.npy')
print("Matriz de similitud cargada (forma:", similarity_matrix.shape, ")")

# Cargar los requisitos originales desde el CSV de los expertos
df = pd.read_csv('data/req_human/checklist_human.csv')
requisitos = df['Descripción'].tolist()
requerimiento_ids = df['ID'].tolist()
print("Requisitos cargados:", len(requisitos))

# Crear una lista de pares con alta similitud
high_similarity_pairs = []
for i in range(len(requisitos)):
    for j in range(i + 1, len(requisitos)):  # Evitar comparar un requisito consigo mismo
        if similarity_matrix[i][j] > 0.7:  # Umbral ajustable (0.7 como ejemplo)
            high_similarity_pairs.append({
                'Req_1_ID': requerimiento_ids[i],
                'Req_1': requisitos[i],
                'Req_2_ID': requerimiento_ids[j],
                'Req_2': requisitos[j],
                'Similarity': similarity_matrix[i][j]
            })

# Convertir a DataFrame y guardar pares con alta similitud
if high_similarity_pairs:
    df_high_similarity = pd.DataFrame(high_similarity_pairs)
    df_high_similarity.to_csv('data/req_human/results/high_similarity_pairs_ex.csv', index=False)
    print("Pares con alta similitud guardados en 'data/req_human/results/high_similarity_pairs_ex.csv':")
    print(df_high_similarity)

    # Mostrar los 3 pares con mayor similitud
    print("\nTop 3 pares con mayor similitud:")
    top_3_pairs = df_high_similarity.sort_values(by='Similarity', ascending=False).head(3)
    print(top_3_pairs)
else:
    print("No se encontraron pares con similitud mayor a 0.7.")

# Definir función para calcular el índice de legibilidad (Flesch Reading Ease)
def get_legibility_score(text):
    """
    Calcula el índice de legibilidad Flesch Reading Ease para un texto en inglés.
    - Retorna un puntaje entre 0 y 100: mayor puntaje = más fácil de leer.
    - Basado en la longitud de palabras y oraciones.
    """
    return textstat.flesch_reading_ease(text)

# Calcular puntuaciones de legibilidad para cada requisito
df['Legibility_Score'] = df['Descripción'].apply(get_legibility_score)
print("Puntuaciones de legibilidad calculadas:")
print(df[['ID', 'Descripción', 'Legibility_Score']].head())  # Mostrar las primeras 5 filas como ejemplo

# Combinar resultados en un DataFrame
df_analysis = pd.DataFrame({
    'ID': requerimiento_ids,
    'Descripción': requisitos,
    'Legibility_Score': df['Legibility_Score'].tolist()  # Usar las puntuaciones calculadas
})

# Añadir la similitud máxima para cada requisito
max_similarity = []
for i in range(len(requisitos)):
    max_sim = 0.0
    similar_id_idx = None
    for j in range(len(requisitos)):
        if i != j and similarity_matrix[i][j] > max_sim:
            max_sim = similarity_matrix[i][j]
            similar_id_idx = j
    similar_id = requerimiento_ids[similar_id_idx] if similar_id_idx is not None else "N/A"
    max_similarity.append(f"{max_sim:.2f}-{similar_id}")

df_analysis['Max_Similarity'] = max_similarity

# Guardar el análisis combinado
df_analysis.to_csv('data/req_human/results/expert_analisis_simiYlegi.csv', index=False)
print("Análisis combinado guardado en 'data/req_human/results/expert_analisis_simiYlegi.csv'")

##PROCESO DE OBTENCION DE RESULTADOS DEL ANALISIS CRUZADO POR GROK ENTRE MODELO DE NOTEBOOK Y AGENTE DE CHATGPT PARA REQUISITOS DE LA IA
# Celda 1: Preprocesar el archivo CSV con corrección de campos
import csv

# Abrir el archivo original y crear uno nuevo
with open('rq5/RQ5-AnalisisCruzadoNote-IA.csv', 'r', encoding='utf-8') as infile, \
     open('rq5/RQ5-AnalisisCruzadoNote-IA_cleaned.csv', 'w', newline='', encoding='utf-8') as outfile:

    reader = csv.reader(infile, quotechar='"', delimiter=',', skipinitialspace=True)
    writer = csv.writer(outfile, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, escapechar='\\')

    # Escribir el encabezado
    header = next(reader)
    writer.writerow(header)

    # Procesar cada línea
    for row in reader:
        if len(row) < 8:
            print(f"Error: Línea con menos de 8 campos: {row}")
            continue
        elif len(row) > 8:
            print(f"Corriendo línea con {len(row)} campos: {row}")
            # Reconstruir los campos de forma dinámica
            id_val = row[0]  # ID
            req_gen = ' '.join(row[1:4]).strip()  # Unir Requisito generado (hasta 3 elementos si es necesario)
            art_fuente = ', '.join(row[4:7]) if len(row) >= 7 else row[4] if len(row) > 4 else ''  # Artículo/Fuente
            extraido = row[7] if len(row) > 7 else 'Sí'  # ¿Extraído correctamente?
            nivel = row[8] if len(row) > 8 else 'Alta'  # Nivel de fidelidad
            tipo_error = row[9] if len(row) > 9 else ''  # Tipo de error
            porc_calidad = row[-1]  # Porcentaje de calidad (último elemento)
            justif = ' '.join(row[10:-1]).strip() if len(row) > 10 else row[10] if len(row) > 10 else ''  # Justificación/Comentarios

            # Asegurarse de que justif no cause error si row[-1] ya fue tomado
            if not justif and len(row) > 10:
                justif = ' '.join(row[10:]).strip().rsplit(' ', 1)[0]  # Tomar todo menos el último (porc_calidad)

            # Escribir la línea corregida
            writer.writerow([id_val, req_gen, art_fuente, extraido, nivel, tipo_error, justif, porc_calidad])
        else:
            writer.writerow(row)

print("Archivo preprocesado guardado como 'rq5/RQ5-AnalisisCruzadoNote-IA_cleaned.csv'")

##CASO PARA REQUISITOS DEL EXPERTO ###RQ5
# Celda: Calcular porcentaje de calidad y generar CSV para requisitos de los expertos con penalización por similitud
import pandas as pd
import numpy as np
import csv  # Importado para usar csv.QUOTE_ALL en la escritura del CSV

# Leer los archivos
# Ajusta las rutas según la ubicación real de tus archivos
# df_notebook contiene las evaluaciones de los expertos (Excel)
df_notebook_experto = pd.read_excel('data-RQ5/req_human/results/Validacion_Requisitos_Expert.xlsx', sheet_name='Sheet1')
# df_llm contiene las métricas de legibilidad y similitud (CSV)
df_llm_experto = pd.read_csv('rq5_results/Result_req_Experto/expert_analisis_simiYlegi.csv', quotechar='"')

# Asegurarse de que ambos DataFrames tengan un ID común para el merge
# Renombrar columnas en df_notebook
df_notebook_experto_cleaned = df_notebook_experto.rename(columns={'ID': 'id'}) if 'ID' in df_notebook_experto.columns else df_notebook_experto
# Renombrar columnas en df_llm (corrigiendo de 'requerimiento_id' a 'ID')
df_llm_experto_cleaned = df_llm_experto.rename(columns={'ID': 'id', 'Max_Similarity': 'similitud_maxima'}) if 'ID' in df_llm_experto.columns else df_llm_experto

# Merge de los DataFrames usando 'id' como clave
# Uso de 'how=left' asegura que se conserven todas las filas de df_notebook
df_merged = pd.merge(df_notebook_experto_cleaned, df_llm_experto_cleaned, on='id', how='left')

# Función para normalizar Legibility_Score a 0-100
def normalize_legibility(score):
    # Verifica si el score es NaN o no es numérico, devuelve 0 en ese caso
    if pd.isna(score) or not isinstance(score, (int, float)):
        return 0
    # Ajusta negativos a 0 y capta valores a 100 (score + 100, max 100)
    return max(0, min(100, score + 100))

# Función para calcular el porcentaje de calidad
def calculate_quality(row):
    # Asigna valor de fidelidad según el nivel reportado
    # Alta = 95%, Media = 70%, Baja = 40% (no aplica en datos actuales)
    fidelity = 95 if row['Nivel de fidelidad'] == 'Alta' else 70 if row['Nivel de fidelidad'] == 'Media' else 40
    
    # Normaliza el Legibility_Score si existe en el row
    legibility = normalize_legibility(row['Legibility_Score']) if 'Legibility_Score' in row else 0
    
    # Bono por ausencia de errores: +20% si no hay error, 0% si hay
    error_bonus = 20 if pd.isna(row['Tipo de error (si aplica)']) or row['Tipo de error (si aplica)'] == '' else 0
    
    # Obtiene el texto de justificación o cadena vacía si no existe
    justification = row['Justificación/Comentarios'] if 'Justificación/Comentarios' in row else ''
    # Bono por justificación: +10% si es positivo, -10% si sugiere mejoras, 0% si neutro
    # Incluye "alineado", "compatible", "fuerte", "coherente" como positivos
    justif_bonus = 10 if any(word in justification.lower() for word in ['positivo', 'clara', 'alineado', 'compatible', 'fuerte', 'coherente']) else \
                   -10 if any(word in justification.lower() for word in ['mejora', 'reformular', 'redundante']) else 0
    
    # Penalización por similitud: -10% si similitud_maxima > 0.8, 0% en caso contrario
    similarity_penalty = -10 if 'similitud_maxima' in row and float(row['similitud_maxima'].split('-')[0]) > 0.8 else 0
    
    # Cálculo del porcentaje de calidad como promedio de los cinco componentes
    quality = (fidelity + legibility + error_bonus + justif_bonus + similarity_penalty) / 5
    return round(quality, 2)

# Aplicar la función para calcular el porcentaje de calidad a todas las filas
df_merged['Porcentaje de calidad'] = df_merged.apply(calculate_quality, axis=1)

# Seleccionar y renombrar columnas para el formato deseado
# Crea una copia del DataFrame con las columnas relevantes
df_result = df_merged[['id', 'Requisito generado', 'Artículo/Fuente', '¿Extraído correctamente?', 
                       'Nivel de fidelidad', 'Tipo de error (si aplica)', 'Justificación/Comentarios', 
                       'Porcentaje de calidad']].copy()
# Renombra 'id' a 'ID' para consistencia con el formato original
df_result = df_result.rename(columns={'id': 'ID'})
# Guardar el resultado en un nuevo CSV
# Mensaje de confirmación y escritura del archivo con formato citado
print("OBTENEMOS FICHERO QUE HACIA CRUCE DE DATOS LEGIBILIDAD Y SIMILITUD + COMENTARIOS EXPERTOS CON PENALIZACIÓN POR SIMILITUD")
df_result.to_csv('RESULTADO_RQ5_EXPERTO_ConSimilitud.csv', index=False, quotechar='"', quoting=csv.QUOTE_ALL, escapechar='\\')

# Mostrar las primeras filas para verificar los resultados
print(df_result.head())

# Celda: Analizar resultados_calidad_requisitos.csv y generar gráfico
import pandas as pd
import matplotlib.pyplot as plt

# Leer el archivo CSV
df = pd.read_csv('RESULTADO_RQ5_EXPERTO_ConSimilitud.csv', quotechar='"')

# Calcular métricas básicas
media_calidad = df['Porcentaje de calidad'].mean()
max_calidad = df['Porcentaje de calidad'].max()
min_calidad = df['Porcentaje de calidad'].min()
requisito_max = df[df['Porcentaje de calidad'] == max_calidad]['ID'].iloc[0]
requisito_min = df[df['Porcentaje de calidad'] == min_calidad]['ID'].iloc[0]

# Resumen de calidad general
print("### Resumen de Calidad de los Requisitos")
print(f"- Media del porcentaje de calidad: {media_calidad:.2f}%")
print(f"- Requisito con mayor calidad ({max_calidad}%): {requisito_max}")
print(f"- Requisito con menor calidad ({min_calidad}%): {requisito_min}")

# Seleccionar los 5 requisitos de mayor y menor calidad
top_5 = df.nlargest(5, 'Porcentaje de calidad')[['ID', 'Porcentaje de calidad']]
bottom_5 = df.nsmallest(5, 'Porcentaje de calidad')[['ID', 'Porcentaje de calidad']]

# Imprimir los datos seleccionados para depuración
print("Top 5:", top_5)
print("Bottom 5:", bottom_5)

# Visualización: Gráfico de barras con los 5 requisitos de mayor y menor calidad
plt.figure(figsize=(10, 5))
plt.bar(top_5['ID'], top_5['Porcentaje de calidad'], color='green', label='Top 5')
plt.bar(bottom_5['ID'], bottom_5['Porcentaje de calidad'], color='red', label='Bottom 5')
plt.axhline(y=media_calidad, color='blue', linestyle='--', label=f'Media: {media_calidad:.2f}%')
plt.title('Top 5 y Bottom 5 Requisitos por Calidad - Requisitos del Experto')
plt.xlabel('ID del Requisito')
plt.ylabel('Porcentaje de Calidad (%)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


##CASO PARA CALCULA CALIDAD LOS REQUISITOS DEL AGENTE
import pandas as pd
import numpy as np
import csv  # Importado para usar csv.QUOTE_ALL en la escritura del CSV

# Leer los archivos
# Ajusta las rutas según la ubicación real de tus archivos
# df_notebook contiene las evaluaciones de los expertos (Excel)
df_notebook_agente = pd.read_excel('rq5_results/Validación_Requisitos_Agente.xlsx', sheet_name='Sheet1')

# df_llm contiene las métricas de legibilidad y similitud (CSV)
df_llm_agente = pd.read_csv('rq5_results/Result_req_Agente/requisitos_analisis_simiYlegi.csv', quotechar='"')

# Renombrar columnas para unificar la clave 'id'
df_notebook_agente_cleaned = df_notebook_agente.rename(columns={'ID': 'id'})
df_llm_agente_cleaned = df_llm_agente.rename(columns={'requerimiento_id': 'id'})

# Merge de los DataFrames usando 'id' como clave
# Uso de 'how=left' asegura que se conserven todas las filas de df_notebook
df_merged_agente = pd.merge(df_notebook_agente_cleaned, df_llm_agente_cleaned, on='id', how='left')

# Función para normalizar Legibility_Score a 0-100
def normalize_legibility(score):
    # Verifica si el score es NaN o no es numérico, devuelve 0 en ese caso
    if pd.isna(score) or not isinstance(score, (int, float)):
        return 0
    # Ajusta negativos a 0 y capta valores a 100 (score + 100, max 100)
    return max(0, min(100, score + 100))

# Función para calcular el porcentaje de calidad
def calculate_quality(row):
    # Asigna valor de fidelidad según el nivel reportado
    # Alta = 95%, Media = 70%, Baja = 40% (no aplica en datos actuales)
    fidelity = 95 if row['Nivel de fidelidad'] == 'Alta' else 70 if row['Nivel de fidelidad'] == 'Media' else 40
    
    # Normaliza el Legibility_Score si existe en el row
    legibility = normalize_legibility(row['legibility_score']) if 'legibility_score' in row else 0
    
    # Bono por ausencia de errores: +20% si no hay error, 0% si hay
    error_bonus = 20 if pd.isna(row['Tipo de error (si aplica)']) or row['Tipo de error (si aplica)'] == '' else 0
    
    # Obtiene el texto de justificación o cadena vacía si no existe
    justification = row['Justificación/Comentarios'] if 'Justificación/Comentarios' in row else ''
    # Bono por justificación: +10% si es positivo, -10% si sugiere mejoras, 0% si neutro
    # Incluye "alineado", "compatible", "fuerte", "coherente" como positivos
    justif_bonus = 10 if any(word in justification.lower() for word in ['positivo', 'clara', 'alineado', 'compatible', 'fuerte', 'coherente']) else \
                   -10 if any(word in justification.lower() for word in ['mejora', 'reformular', 'redundante']) else 0
    
    # Penalización por similitud: -10% si similitud_maxima > 0.8, 0% en caso contrario
    similarity_penalty = -10 if 'similitud_maxima' in row and float(row['similitud_maxima'].split('-')[0]) > 0.8 else 0
    
    # Cálculo del porcentaje de calidad como promedio de los cinco componentes
    quality = (fidelity + legibility + error_bonus + justif_bonus + similarity_penalty) / 5
    return round(quality, 2)

# Aplicar la función para calcular el porcentaje de calidad a todas las filas
df_merged_agente['Porcentaje de calidad'] = df_merged_agente.apply(calculate_quality, axis=1)

# Seleccionar y renombrar columnas para el formato deseado
# Crea una copia del DataFrame con las columnas relevantes
df_result_agente = df_merged_agente[['id', 'Requisito generado', 'Artículo/Fuente', '¿Extraído correctamente?', 
                       'Nivel de fidelidad', 'Tipo de error (si aplica)', 'Justificación/Comentarios', 
                       'Porcentaje de calidad']].copy()
# Renombra 'id' a 'ID' para consistencia con el formato original
df_result_agente = df_result_agente.rename(columns={'id': 'ID'})
# Guardar el resultado en un nuevo CSV
# Mensaje de confirmación y escritura del archivo con formato citado
print("OBTENEMOS FICHERO QUE HACIA CRUCE DE DATOS LEGIBILIDAD Y SIMILITUD + COMENTARIOS AGENTE CON PENALIZACIÓN POR SIMILITUD")
df_result_agente.to_csv('RESULTADO_RQ5_AGENTE_ConSimilitud.csv', index=False, quotechar='"', quoting=csv.QUOTE_ALL, escapechar='\\')

# Mostrar las primeras filas para verificar los resultados
print(df_result_agente.head())


# Celda: Analizar resultados_calidad_requisitos.csv y generar gráfico
import pandas as pd
import matplotlib.pyplot as plt

# Leer el archivo CSV
df = pd.read_csv('RESULTADO_RQ5_AGENTE_ConSimilitud.csv', quotechar='"')

# Calcular métricas básicas
media_calidad = df['Porcentaje de calidad'].mean()
max_calidad = df['Porcentaje de calidad'].max()
min_calidad = df['Porcentaje de calidad'].min()
requisito_max = df[df['Porcentaje de calidad'] == max_calidad]['ID'].iloc[0]
requisito_min = df[df['Porcentaje de calidad'] == min_calidad]['ID'].iloc[0]

# Resumen de calidad general
print("### Resumen de Calidad de los Requisitos")
print(f"- Media del porcentaje de calidad: {media_calidad:.2f}%")
print(f"- Requisito con mayor calidad ({max_calidad}%): {requisito_max}")
print(f"- Requisito con menor calidad ({min_calidad}%): {requisito_min}")

# Seleccionar los 5 requisitos de mayor y menor calidad
top_5 = df.nlargest(5, 'Porcentaje de calidad')[['ID', 'Porcentaje de calidad']]
bottom_5 = df.nsmallest(5, 'Porcentaje de calidad')[['ID', 'Porcentaje de calidad']]

# Imprimir los datos seleccionados para depuración
print("Top 5:", top_5)
print("Bottom 5:", bottom_5)

# Visualización: Gráfico de barras con los 5 requisitos de mayor y menor calidad
plt.figure(figsize=(10, 5))
plt.bar(top_5['ID'], top_5['Porcentaje de calidad'], color='green', label='Top 5')
plt.bar(bottom_5['ID'], bottom_5['Porcentaje de calidad'], color='red', label='Bottom 5')
plt.axhline(y=media_calidad, color='blue', linestyle='--', label=f'Media: {media_calidad:.2f}%')
plt.title('Top 5 y Bottom 5 Requisitos por Calidad - Requisitos del AGENTE')
plt.xlabel('ID del Requisito')
plt.ylabel('Porcentaje de Calidad (%)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


##ANALISIS
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# Leer los datos reales desde los archivos CSV
agente_df = pd.read_csv('rq5_results/Result_req_Agente/high_similarity_pairs_agente.csv')
experto_df = pd.read_csv('rq5_results/Result_req_Experto/high_similarity_pairs_experto.csv')

# Leer los datos de calidad desde los archivos proporcionados
sir_df = pd.read_csv('RESULTADO_RQ5_AGENTE_ConSimilitud.csv')
sec_df = pd.read_csv('RESULTADO_RQ5_EXPERTO_ConSimilitud.csv')

# 1. Preparar datos para los charts
# Distribución de porcentajes de calidad basada en los archivos reales
score_ranges = ["32-35", "35-38", "38-41", "41-44", "44-46"]
sec_data = [len(sec_df[(sec_df['Porcentaje de calidad'] >= 32) & (sec_df['Porcentaje de calidad'] < 35)]),
            len(sec_df[(sec_df['Porcentaje de calidad'] >= 35) & (sec_df['Porcentaje de calidad'] < 38)]),
            len(sec_df[(sec_df['Porcentaje de calidad'] >= 38) & (sec_df['Porcentaje de calidad'] < 41)]),
            len(sec_df[(sec_df['Porcentaje de calidad'] >= 41) & (sec_df['Porcentaje de calidad'] < 44)]),
            len(sec_df[(sec_df['Porcentaje de calidad'] >= 44) & (sec_df['Porcentaje de calidad'] <= 46)])]
sir_data = [len(sir_df[(sir_df['Porcentaje de calidad'] >= 32) & (sir_df['Porcentaje de calidad'] < 35)]),
            len(sir_df[(sir_df['Porcentaje de calidad'] >= 35) & (sir_df['Porcentaje de calidad'] < 38)]),
            len(sir_df[(sir_df['Porcentaje de calidad'] >= 38) & (sir_df['Porcentaje de calidad'] < 41)]),
            len(sir_df[(sir_df['Porcentaje de calidad'] >= 41) & (sir_df['Porcentaje de calidad'] < 44)]),
            len(sir_df[(sir_df['Porcentaje de calidad'] >= 44) & (sir_df['Porcentaje de calidad'] <= 46)])]

# Sugerencias de mejora (aproximadas, basadas en requisitos con nivel de fidelidad 'Media')
improvement_data = {"Expertos (SEC)": len(sec_df[sec_df['Nivel de fidelidad'] == 'Media']),
                    "Agente (SIR)": len(sir_df[sir_df['Nivel de fidelidad'] == 'Media'])}

# Distribución de similitudes
def get_similarity_distribution(df, group_name):
    similarities = df['Similarity'].values
    bins = [0.7, 0.75, 0.8, 0.85, 0.9, 1.0]
    hist, _ = np.histogram(similarities, bins=bins)
    return hist

sec_sim_data = get_similarity_distribution(experto_df, "Expertos (SEC)")
sir_sim_data = get_similarity_distribution(agente_df, "Agente (SIR)")
similarity_ranges = ["0.7-0.75", "0.75-0.8", "0.8-0.85", "0.85-0.9", "0.9-1.0"]

# 2. Análisis básico con matplotlib
plt.figure(figsize=(15, 5))

# Chart 1: Distribución de porcentajes de calidad
plt.subplot(1, 3, 1)
bar_width = 0.35
x = np.arange(len(score_ranges))
plt.bar(x - bar_width/2, sec_data, bar_width, color='#FF6384', alpha=0.7, label='Expertos (SEC)')
plt.bar(x + bar_width/2, sir_data, bar_width, color='#36A2EB', alpha=0.7, label='Agente (SIR)')
plt.xlabel('Rango de porcentaje de calidad')
plt.ylabel('Número de requisitos')
plt.title('Distribución de porcentajes de calidad')
plt.xticks(x, score_ranges)
plt.legend()

# Chart 2: Frecuencia de sugerencias de mejora
plt.subplot(1, 3, 2)
plt.bar(improvement_data.keys(), improvement_data.values(), color=['#FF6384', '#36A2EB'])
plt.title('Frecuencia de sugerencias de mejora')
plt.xlabel('Grupo')
plt.ylabel('Número de requisitos')

# Chart 3: Distribución de similitudes
plt.subplot(1, 3, 3)
bar_width = 0.35
x = np.arange(len(similarity_ranges))
plt.bar(x - bar_width/2, sec_sim_data, bar_width, color='#FF6384', alpha=0.7, label='Expertos (SEC)')
plt.bar(x + bar_width/2, sir_sim_data, bar_width, color='#36A2EB', alpha=0.7, label='Agente (SIR)')
plt.xlabel('Rango de similitud')
plt.ylabel('Número de pares')
plt.title('Distribución de similitudes entre pares')
plt.xticks(x, similarity_ranges)
plt.legend()

plt.tight_layout()
plt.show()


# 3. Resumen estadístico
print("Estadísticas preliminares:")
print(f"Expertos (SEC): Número de pares con similitud >0.7: {len(experto_df)}, Promedio similitud: {experto_df['Similarity'].mean():.3f}")
print(f"Agente (SIR): Número de pares con similitud >0.7: {len(agente_df)}, Promedio similitud: {agente_df['Similarity'].mean():.3f}")